In [42]:
import pandas as pd
import time
import random
import json
from datetime import datetime
from pytz import timezone

# scrapping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import bs4
import requests
from requests.exceptions import RequestException


import warnings
warnings.filterwarnings('ignore')

## Ingreso a la página

In [1]:
url = 'https://epinvestiga.com/'

In [3]:
response = requests.get(url, proxies={'http': None, 'https': None}, timeout=60)

In [5]:
response.content

b'<!DOCTYPE html>\n<!--[if lt IE 7]>\n<html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="es"><![endif]-->\n<!--[if IE 7]>\n<html class="no-js lt-ie9 lt-ie8" lang="es"><![endif]-->\n<!--[if IE 8]>\n<html class="no-js lt-ie9" lang="es"><![endif]-->\n<!--[if IE 9]>\n<html class="no-js lt-ie10" lang="es"><![endif]-->\n<!--[if gt IE 9]><!-->\n<html class="no-js" lang="es">\n<!--<![endif]-->\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/>\n\t<meta charset="UTF-8">\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0">\n\t<link rel="profile" href="https://gmpg.org/xfn/11">\n\t<link rel="pingback" href="https://epinvestiga.com/xmlrpc.php">\n\n\t<!-- Fav and touch icons -->\n\t\t\t<link rel="shortcut icon" href="https://epinvestiga.com/wp-content/uploads/2024/03/favicon-epi-3.png">\n\t\t\t\t<link rel="apple-touch-icon-precomposed" href="https://epinvestiga.com/wp-content/uploads/2024/03/57x57.png">\n\t\t\t\t<link rel="apple-touch-icon-precompos

In [6]:
soup = BeautifulSoup(response.content, 'html.parser')

In [7]:
with open('../data/epInvestiga_home.html', 'w') as ep:
    ep.write(soup.prettify())

## Probando buscar

In [14]:
nombre = 'Bernardo arevalo'
nombre = nombre.split()
nombre = '+'.join(nombre)

busqueda = f'?s={nombre}'

In [17]:
response = requests.get(url + busqueda, proxies={'http': None, 'https': None}, timeout=60)

In [18]:
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    with open('../data/epInvestiga_busqueda1.html', 'w') as ep:
        ep.write(soup.prettify())

In [19]:
page_numbers = soup.find_all('a', class_='page-numbers')

page_numbers = [int(element.text.strip()) for element in page_numbers if element.text.strip().isdigit()]

In [21]:
max_page = max(page_numbers) if page_numbers else 1

In [23]:
type(soup)

bs4.BeautifulSoup

### extraemos todos los artículos de una sola pagina

In [44]:
def dividir_fecha_hora_iso(fecha_iso):
    # Parsear la fecha ISO
    fecha = datetime.fromisoformat(fecha_iso)
    
    # Asegurar que está en zona horaria de Centroamérica (GMT-6)
    centroamerica_tz = timezone("America/Guatemala")
    fecha_gmt6 = fecha.astimezone(centroamerica_tz)
    
    # Formatear fecha y hora
    fecha_formateada = fecha_gmt6.strftime("%Y-%m-%d")  # YYYY-MM-DD
    hora_formateada = fecha_gmt6.strftime("%H:%M:%S")  # HH:MM:SS
    
    # Retornar el diccionario
    return {
        "fecha": fecha_formateada,
        "hora": hora_formateada
    }

def extraer_info_articulo(soup : bs4.BeautifulSoup) -> dict:
    # link del articulo
    link = soup.find("a", class_="element-wrap")["href"]

    # titulo del articulo
    titulo = soup.find("h3", class_="post-title").get_text(strip=True)

    # extraer el resumen
    resumen = soup.find("div", class_="entry-summary").get_text(strip=True)

    # fecha de entrada
    fecha_entrada = soup.find("time", class_="entry-date")["datetime"]

    # fecha actualizacion
    fecha_actualizacion = soup.find("time", class_="modify-date")
    if fecha_actualizacion:
        fecha_actualizacion = fecha_actualizacion["datetime"]
    else:
        fecha_actualizacion = None

    # autor
    autor = soup.find("div", class_="post-footer").find("span").get_text(strip=True)

    return {
        "link": link,
        "titulo": titulo,
        "resumen": resumen,
        "fecha_entrada": dividir_fecha_hora_iso(fecha_entrada),
        "fecha_actualizacion": dividir_fecha_hora_iso(fecha_actualizacion),
        "autor": autor,
    }

In [27]:
articulos = soup.find_all("article", class_="post-item")
resultado = [extraer_info_articulo(articulo) for articulo in articulos]

In [28]:
data = {"page": 1, "articulos": resultado}

In [30]:
formateado = json.dumps(data, indent=4, ensure_ascii=False)
with open("../data/epInvestiga/resultados_1.json", 'w', encoding='utf-8') as f:
    f.write(formateado)

## ahora para todas las paginas

In [45]:
full = []
print('Inicio...')
for i in range(1, max_page + 1):
    # agregamos el número de página
    page = f'page/{i}/'
    print(f'Solicitando info pag {i}')
    
    try:
        # solicitamos la información de la pag
        response = requests.get(
            url + page + busqueda, 
            proxies={'http': None, 'https': None}, 
            timeout=60,
            verify=False)
        print(f'Estado: {response.status_code}')

        # si se recibio respuesta correcta, parseamos
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # extraemos la información de los articulos
            articulos = soup.find_all("article", class_="post-item")
            resultado = [extraer_info_articulo(articulo) for articulo in articulos]

            # juntamos la data en formato json
            data = {"page": i, "articulos": resultado}
            full.append(data)
    except RequestException as e:
        print(f'Error al procesar pagina {i}: {e}')
        continue


info = json.dumps({'epInvestiga': full}, indent=4, ensure_ascii=False)
with open(f"../data/epInvestiga/resultados_1_{nombre}.json", 'w', encoding='utf-8') as f:
    f.write(info)

Inicio...
Solicitando info pag 1
Estado: 200
Solicitando info pag 2
Estado: 200
Solicitando info pag 3
Estado: 200
Solicitando info pag 4
Estado: 200
Solicitando info pag 5
Estado: 200
Solicitando info pag 6
Estado: 200
Solicitando info pag 7
Estado: 200
Solicitando info pag 8
Estado: 200
Solicitando info pag 9
Estado: 200
Solicitando info pag 10
Estado: 200
Solicitando info pag 11
Estado: 200
Solicitando info pag 12
Estado: 200
Solicitando info pag 13
Estado: 200
Solicitando info pag 14
Estado: 200
Solicitando info pag 15
Estado: 200
Solicitando info pag 16
Estado: 200
Solicitando info pag 17
Estado: 200
Solicitando info pag 18
Error al procesar pagina 18: HTTPSConnectionPool(host='epinvestiga.com', port=443): Max retries exceeded with url: /page/18/?s=Bernardo+arevalo (Caused by SSLError(SSLError(1, '[SSL] record layer failure (_ssl.c:1020)')))
Solicitando info pag 19
Estado: 200
Solicitando info pag 20
Estado: 200
Solicitando info pag 21
Estado: 200
Solicitando info pag 22
Estado: 2